# Collecting Reddit submissions

## Libraries

In [ ]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import regex as re
import sqlite3

import utils
from params import sql_db

## Setup

In [ ]:
# Define search parameters
# the keys are the parameter names (see https://pushshift.io/api-parameters/ for possible parameters)
param_dict = {'subreddit':'movies',
              'title':"Official Discussion",
              'size':1000, # 1000 is the maximum number that can be collected per single request. No reason to change this.
             }

# Keys to collect from submissions
submission_keys = ('id', 'title', 'score', 'num_comments', 'url', 'created_utc')

# Define submission_limit, the number of submissions to be obtained by the API
submission_limit = 100000

# Define location and name of SQL database, create a connection object
conn = sqlite3.connect(sql_db)

## Create SQL database

In [ ]:
# Create SQL table for submissions if it does not yet exist
sql_create_submissions_table = """ CREATE TABLE IF NOT EXISTS submissions (
                                    submission_id text PRIMARY KEY,
                                    title text,
                                    score integer,
                                    num_comments integer,
                                    url text,
                                    created integer
                                ); """

with conn:
    utils.interact_with_db(conn, sql_create_submissions_table)

## Find oldest collected submission

In [ ]:
# In order to add more submissions:
try:
    with conn:
        param_dict['before'] = utils.interact_with_db(conn, 
                                                      '''SELECT min(created) FROM submissions;''', 
                                                      fetch='cur.fetchone()[0]')
        
        print(f"Oldest submission found is from {datetime.fromtimestamp(param_dict['before'])}")
except:
    print("Didn't find anything. Either there is no data or something went wrong")

## Collect submissions

In [ ]:
# Based on: https://github.com/SeyiAgboola/Reddit-Data-Mining/blob/master/Using_Pushshift_Module_to_extract_Submissions.ipynb
print(f"Starting at {datetime.now()}")
sub_count = 0
n_submission_keys = len(submission_keys)

# Collect first set of submissions
# We need to run this function outside the loop first to get the updated before variable
data = utils.get_pushshift_data(param_dict)

print(f"The youngest submission that fits the criteria is from: {datetime.fromtimestamp(data[0]['created_utc'])}")

with tqdm(total=submission_limit) as pbar:
    while len(data) > 0: 
        if sub_count < submission_limit:
            
            # Select relevant data from each submission in data 
            submissions_data = [utils.collect_submission_data(submission, keys=submission_keys) for submission in data]
            
            # Add data to database
            with conn:
                n_submissions = utils.add_rows(conn, 'submissions', n_submission_keys, submissions_data)
                
            # Update counter and tqdm
            sub_count+= n_submissions
            pbar.update(n_submissions)
                
            # Set the new 'before' parameter
            param_dict['before'] = data[-1]['created_utc']
            
            # Collect next set of submissions
            data = utils.get_pushshift_data(param_dict)
            
        else:
            print(f"Reached submission limit at {datetime.now()}")
            print(f"Didn't collect submissions posted before {datetime.fromtimestamp(param_dict['before'])}")
            break
    

print(f"Finished at {datetime.now()}")